# User Engagement WIP....

The purpose of this notebook is to identify areas where customers ....

Using this notebook, you'll be able to:
- Identify ...
- Analyze....
- Recommend ....

### How to use this notebook

Before doing anything, **make a copy** of this notebook in your personal temp space. Don't modify the main copy, as it will be used by other SAs and changes could overwrite each other.

Once you've made a copy, you can modify variables and set up the environment for your customer using the sections below:

1. Run the `init_` cells to create connections and variables.
2. Use the `config_` section to set the notebook context for your customer account.
3. Run the `cache_` cells to generate all the data needed for the analysis.

The notebook then begins in earnest with the `Overview` section.

### Sharing with customers

All the queries used in this notebook are built using customer-visible data, assuming they are in the PrPr for warehouse utilization views. They can be freely shared with customers to enable them to do their own analysis. To share, simply replace `PST.SVCS` with `SNOWFLAKE.ACCOUNT_USAGE` in all the scripts. If any script requires further customization, it will be explained in that section.

**Note:** All the queries will focus on an Account, rather than an Organization, because that is how the relevant metrics are presented to the customer in the product.


In [ ]:
--use role technical_account_manager;
use role SUPPORT_ENGINEER;
use database snowhouse_import;
--use warehouse doubleblackdiamonds;

In [ ]:
import streamlit as st
import pandas as pd
import altair as alt
import plotly.graph_objects as go

#from streamlit_extras import stylable_container
from snowflake.snowpark.context import get_active_session
from datetime import datetime, date, time
session = get_active_session()

In [ ]:
set top_x_wh = 30;
set lookback_days = 15;
set start_time = dateadd(day,-$lookback_days,current_date);

### Find Your Account

You will need to set the notebook context for the account you wish to analyze. Enter a name or search value into `config_acct_set` and run it, then run `config_acct_search`. It will return all matching results from account names, aliases, organizations, and salesforce names. The results are sorted by most recent credit usage, so your account is likely in the top few results. If it's not, try a different search value.

If you already know your Account ID and Deployment, you may skip the search steps.

In [ ]:
# import pandas as pd
# import streamlit as st
# session = get_active_session()
@st.cache_data(ttl=1600)
def fetch_priority_accounts(_session):
    query = """
        SELECT NAME, ORG_ID_C 
        FROM FIVETRAN.SALESFORCE.ACCOUNT 
        WHERE PRIORITY_SUPPORT_C = TRUE AND IS_DELETED = FALSE
        ORDER BY NAME
    """
    return pd.DataFrame(session.sql(query).collect())

@st.cache_data(ttl=1600)
def fetch_accounts_by_orgid(_session, org_id):
    query = f"""
        SELECT ID, ALIAS, NAME, DEPLOYMENT
        FROM SNOWHOUSE_IMPORT.PROD.ACCOUNT_ETL_V
        WHERE REPLICATION_GROUP = '{org_id}' ORDER BY NAME
    """
    return pd.DataFrame(session.sql(query).collect())


st.write("**************")
priority_accounts = fetch_priority_accounts(session)
customer_name = st.selectbox(
    "Please Select Your Priority Support Customer", 
    priority_accounts['NAME'])

org_id = priority_accounts.loc[priority_accounts['NAME'] == customer_name, 'ORG_ID_C'].values[0]
st.write("Selected ORG_ID_C:", org_id)
st.write("**************")
# accounts = fetch_accounts_by_orgid(session, org_id)
# st.write(accounts)
# account_name = st.selectbox(
#     "Please Select The Snowflake Account", 
#     accounts['NAME'].unique())
# st.write("**************")
# account_rows = accounts[accounts['NAME'] == account_name]

# if len(account_rows) > 1:
#     deployment = st.selectbox("Please Select The Snowflake Deployment", account_rows['DEPLOYMENT'])
#     selected_row = account_rows[account_rows['DEPLOYMENT'] == deployment].iloc[0]
# else:
#     selected_row = account_rows.iloc[0]

# acc_id = selected_row['ID']
# acc_id = int(acc_id)
# acc_dep = selected_row['DEPLOYMENT']
# #acc_dep = acc_dep.iloc[0]
# acc_name = selected_row['NAME']
# #acc_name = acc_name
# st.write(f"Account ID: {acc_id}, Deployment: {acc_dep}, Name: {acc_name}")

In [ ]:
accounts = fetch_accounts_by_orgid(session, org_id)
st.write(accounts)
account_name = st.selectbox(
    "Please Select The Snowflake Account", 
    accounts['NAME'].unique())
st.write("**************")
account_rows = accounts[accounts['NAME'] == account_name]

if len(account_rows) > 1:
    deployment = st.selectbox("Please Select The Snowflake Deployment", account_rows['DEPLOYMENT'])
    selected_row = account_rows[account_rows['DEPLOYMENT'] == deployment].iloc[0]
else:
    selected_row = account_rows.iloc[0]

acc_id = selected_row['ID']
acc_id = int(acc_id)
acc_dep = selected_row['DEPLOYMENT']
#acc_dep = acc_dep.iloc[0]
acc_name = selected_row['NAME']
#acc_name = acc_name
st.write(f"Account ID: {acc_id}, Deployment: {acc_dep}, Name: {acc_name}")

In [ ]:
set acct_search = '{{acc_name}}';
--select '{{accName}};
--set acct_search = 'EY13321';

select
    m.salesforce_account_name,
    m.organization_name,
    m.snowflake_account_name,
    m.snowflake_account_alias,
    m.snowflake_deployment,
    m.snowflake_account_id,
    round(u.total_credits) daily_credits,
    round(u.daily_storage_tb) daily_storage_tb,
from finance.customer.salesforce_snowflake_mapping m
join finance.customer.usage_daily u on m.snowflake_deployment = u.snowflake_deployment and m.snowflake_account_id = u.snowflake_account_id
where (m.salesforce_account_name ilike '%'|| $acct_search ||'%' or m.snowflake_account_name ilike '%'|| $acct_search ||'%' or m.organization_name ilike '%'|| $acct_search ||'%')
  and u.usage_date = dateadd(day,-1,current_date)
order by daily_credits desc nulls last;

### Set Account ID and Deployment

When you've identified your account, copy __snowflake_account_id__ and __snowflake_deployment__ from the results above into `config_acct_context` and run it.

In [ ]:
--call pst.svcs.sp_set_account_context (329091,'va2'); --Snowhouse

--call pst.svcs.sp_set_account_context (4329,'va');--Fidelity Production
--call pst.svcs.sp_set_account_context (192678,'azeastus2prod'); --Carmax
--call pst.svcs.sp_set_account_context (263303,'va3'); --Cargill, Inc263303
--call pst.svcs.sp_set_account_context (98070,'gcpuseast4'); --Solenis, Prod
call pst.svcs.sp_set_account_context ({{acc_id}},'{{acc_dep}}');

### Cache Data

Run the cells below to query `QUERY_HISTORY` and `SESSIONS` views, and cache the data we'll be using for analysis.

In [ ]:
-- Set your context variables as appropriate:
--set ACCOUNT_ID = <customer_account_id>;
--set ACCOUNT_ID = {{accId}};

-- Get user query and object access metrics for last 30 days:
with user_queries as (
  select
    user_name,
    count(*) as total_queries,
    max(start_time) as last_query_time
  from
    pst.svcs.QUERY_HISTORY
  where
    start_time >= dateadd('day', -30, current_timestamp())
    --and account_id = $ACCOUNT_ID
  group by
    user_name
),
object_access as (
  select
    user_name,
    count(distinct BASE_OBJECTS_ACCESSED) as objects_accessed
  from
    pst.svcs.ACCESS_HISTORY
  where
    query_start_time >= dateadd('day', -30, current_timestamp())
    --and account_id = $ACCOUNT_ID
  group by
    user_name
)
select
  q.user_name,
  q.total_queries,
  nvl(a.objects_accessed, 0) as objects_accessed,
  q.last_query_time
from
  user_queries q
  left join object_access a on q.user_name = a.user_name
order by
  q.total_queries desc;

In [ ]:
UserQandO = UserEngagement.to_pandas()
#st.bar_chart(UserQandO, x="USER_NAME", y=["TOTAL_QUERIES","OBJECTS_ACCESSED"],horizontal = True)
st.bar_chart(UserQandO, x="USER_NAME", y=["TOTAL_QUERIES","OBJECTS_ACCESSED"],horizontal = True)


In [ ]:
-- Set your context variables as appropriate:
-- set ACCOUNT_ID = <customer_account_id>;

-- Get user query and object access metrics for last 30 days:

select
QUERY_TYPE, 
DATABASE_NAME, 
SCHEMA_NAME, 
USER_NAME, 
ROLE_NAME, 
WAREHOUSE_NAME,
WAREHOUSE_SIZE, 
SUM(TOTAL_ELAPSED_TIME) as TOTAL_ELAPSED_TIME,
SUM(COMPILATION_TIME) as TOTAL_COMPILATION_TIME, 
SUM(EXECUTION_TIME) as TOTAL_EXECUTION_TIME,
COUNT(QUERY_ID) as TOTAL_QUERIES,
COUNT(SESSION_ID) TOTAL_SESSIONS
from
    pst.svcs.QUERY_HISTORY
where
    start_time >= dateadd('day', -30, current_timestamp())
    AND WAREHOUSE_NAME NOT ILIKE 'COMPUTE_%'
    --and account_id = $ACCOUNT_ID
group by ALL;


In [ ]:
SessStAllx = UserEngagementQueries.to_pandas()
SessStAll = SessStAllx.fillna("Empty")
SessSt = UserEngagementQueries.to_pandas()


#st.write(SessSt)

#SessSt = SessStAll.loc[SessStAll['SCHEMA_NAME'].str.contains('WEATHER')]

SessionsCount  = SessSt.groupby(['SCHEMA_NAME', 'ROLE_NAME', 'USER_NAME', 'QUERY_TYPE'],as_index=False).agg({'TOTAL_SESSIONS':['sum'],'TOTAL_QUERIES': ['mean','min','max','sum']})
SessionsCount.columns = ['_'.join(col) for col in SessionsCount.columns.values]
SessionsCount.rename(columns={"ROLE_NAME_": "ROLE_NAME"},inplace=True)
SessionsCount.rename(columns={"USER_NAME_": "USER_NAME"},inplace=True)
SessionsCount.rename(columns={"QUERY_TYPE_": "QUERY_TYPE"},inplace=True)
SessionsCount.rename(columns={"SCHEMA_NAME_": "SCHEMA_NAME"},inplace=True)
st.write(SessionsCount)



# 1. Create a sample DataFrame
# data = {
#     'Source_Category': ['A', 'A', 'B', 'B', 'C', 'C'],
#     'Target_Category': ['X', 'Y', 'X', 'Z', 'Y', 'Z'],
#     'Value': [10, 5, 8, 12, 7, 3]
# }
df = pd.DataFrame(SessionsCount)

# 2. Prepare data for Sankey diagram
# Create a list of unique labels from 'Source_Category' and 'Target_Category'
labels = list(pd.concat([df['ROLE_NAME'], df['SCHEMA_NAME']]).unique())


# Map labels to indices, as Plotly's Sankey expects numerical indices for source and target
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create source, target, and value lists based on the DataFrame
sources = df['ROLE_NAME'].map(label_to_index).tolist()
targets = df['SCHEMA_NAME'].map(label_to_index).tolist()
values = df['TOTAL_QUERIES_sum'].tolist()

# 3. Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,  # Use the unique labels for node labels
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
    )
)])

fig.update_layout(title_text="Sample Sankey Diagram Role Sessions by Warehouse",title_font_color="Black", font_size=12,font_color="black",)

# fig.update_layout(
#     title_text="Basic Sankey Diagram",
#     font_family="Courier New",
#     font_color="blue",
#     font_size=12,
#     title_font_family="Times New Roman",
#     title_font_color="red",
# )
# Display in Streamlit
st.plotly_chart(fig)


In [ ]:
-- Set your context variables as appropriate:
-- set ACCOUNT_ID = <customer_account_id>;

-- Get user query and object access metrics for last 30 days:

select
QUERY_TEXT,
QUERY_TAG,
QUERY_TYPE,
SESSION_ID,
DATABASE_NAME,
SCHEMA_NAME, 
USER_NAME, 
ROLE_NAME, 
WAREHOUSE_NAME,
WAREHOUSE_SIZE, 
is_client_generated_statement,
transaction_id,
SUM(TOTAL_ELAPSED_TIME) as TOTAL_ELAPSED_TIME,
SUM(COMPILATION_TIME) as TOTAL_COMPILATION_TIME, 
SUM(EXECUTION_TIME) as TOTAL_EXECUTION_TIME,
COUNT(QUERY_PARAMETERIZED_HASH) as TOTAL_QUERIES
from
    pst.svcs.QUERY_HISTORY
where
    start_time >= dateadd('day', -30, current_timestamp())
    AND WAREHOUSE_NAME NOT ILIKE 'COMPUTE_%'
    --and account_id = $ACCOUNT_ID
group by ALL;



In [ ]:
AllQ = AllQueriesText.to_pandas()

SessionsCount  = AllQ.groupby(['SESSION_ID','WAREHOUSE_NAME'],as_index=False).agg({'TOTAL_QUERIES':['sum'],'TOTAL_QUERIES': ['mean','min','max','sum']})
SessionsCount.columns = ['_'.join(col) for col in SessionsCount.columns.values]
SessionsCount.rename(columns={"SESSION_ID_": "SESSION_ID"},inplace=True)
SessionsCount.rename(columns={"WAREHOUSE_NAME_": "WH"},inplace=True)
st.write(SessionsCount)

points = alt.Chart(SessionsCount).mark_point().encode(
             x='WH',
             y='TOTAL_QUERIES_sum',
             color='SESSION_ID',
             size = 'TOTAL_QUERIES_sum',
             tooltip=[
                 alt.Tooltip("SESSION_ID", title="SESSION_ID"),
                 alt.Tooltip("WH", title="WH"),
             ],
         ).interactive()

chart = alt.vconcat(points, data=SessionsCount, title="Session Queries")
st.altair_chart(chart, theme="streamlit", use_container_width=True)

#st.write(AllQ)

In [ ]:
df = UserEngagementQueries.to_pandas()

#chart_data=pd.DataFrame(data=pd.pivot_table(df, index=['USER_NAME'], values=['TOTAL_QUERIES'], aggfunc='sum'))
#st.write(chart_data)
#st.bar_chart(chart_data)

queryCountH_df  = df.groupby(['WAREHOUSE_NAME','USER_NAME'],as_index=False).agg({'TOTAL_QUERIES':['sum'],'TOTAL_QUERIES': ['mean','min','max','sum']})
queryCountH_df.columns = ['_'.join(col) for col in queryCountH_df.columns.values]
queryCountH_df.rename(columns={"USER_NAME_": "USER_NAME"},inplace=True)
queryCountH_df.rename(columns={"WAREHOUSE_NAME_": "WH"},inplace=True)
st.write(queryCountH_df)

points = alt.Chart(queryCountH_df).mark_point().encode(
             x='WH',
             y='TOTAL_QUERIES_sum',
             color='USER_NAME',
             size = 'TOTAL_QUERIES_sum',
             tooltip=[
                 alt.Tooltip("USER_NAME", title="USER_NAME"),
                 alt.Tooltip("WH", title="WH"),
             ],
         ).interactive()

chart = alt.vconcat(points, data=queryCountH_df, title="USER_NAME Cost")
st.altair_chart(chart, theme="streamlit", use_container_width=True)

In [ ]:
SELECT
*
FROM
    pst.svcs.METERING_DAILY_HISTORY
WHERE 1=1
    --SERVICE_TYPE != 'WAREHOUSE_METERING'
    AND USAGE_DATE >= DATEADD(month, -1, CURRENT_DATE)
ORDER BY
    USAGE_DATE DESC;

In [ ]:
CredPServ = CreditsPerService.to_pandas()
st.bar_chart(CredPServ, x="USAGE_DATE", y="CREDITS_USED", color="SERVICE_TYPE",horizontal = True)


# Create an Altair bar chart
chart = alt.Chart(CredPServ).mark_bar(size=40).encode(
    x='USAGE_DATE',
    y='CREDITS_USED',
    color = 'SERVICE_TYPE'
)


# Display the chart in Streamlit
st.altair_chart(chart, use_container_width=True)

In [ ]:
select 
    qh.USER_NAME, 
    --qh.ROLE_NAME,
    qh.query_type,
    --se.CLIENT_APPLICATION_ID, 
    --se.CLIENT_APPLICATION_VERSION, 
    --se.CLIENT_ENVIRONMENT,
    --COALESCE(se.CLIENT_ENVIRONMENT:APPLICATION::varchar) AS app,
    PARSE_JSON(se.CLIENT_ENVIRONMENT):APPLICATION AS app,
    COUNT (distinct qh.SESSION_ID) as TOTAL_SESSIONS,
    
    --count(distinct qh.query_text),
    --count(distinct qh.USER_NAME) as users_per_session
from
    pst.svcs.QUERY_HISTORY qh
    JOIN pst.svcs.SESSIONS se on (qh.SESSION_ID = se.SESSION_ID)
WHERE 
qh.start_time >= dateadd('day', -3, current_timestamp())
GROUP BY ALL ;
-- where
--     start_time >= dateadd('day', -30, current_timestamp());


In [ ]:

# from streamlit_extras.stylable_container import stylable_container
# with stylable_container(
#     key="my_styled_container",
#     css_styles="""
#     {
#         border: 2px solid green;
#         border-radius: 10px;
#         padding: 20px;
#         .node-label-text-path {
#         fill: rgb(250, 250, 250) !important;
#         text-shadow: none
#         }
#     }
#     """,
# ):
#     st.write("This text is inside a styled container.")

# st.write("This text is outside the styled container.")

appSess = AppSessions.to_pandas()


appSess_df  = appSess.groupby(['APP','USER_NAME','QUERY_TYPE'],as_index=False).agg({'TOTAL_SESSIONS':['sum'],'TOTAL_SESSIONS': ['mean','min','max','sum']})
appSess_df.columns = ['_'.join(col) for col in appSess_df.columns.values]
appSess_df.rename(columns={"USER_NAME_": "USER_NAME"},inplace=True)
appSess_df.rename(columns={"APP_": "APP"},inplace=True)
appSess_df.rename(columns={"QUERY_TYPE_": "QUERY_TYPE"},inplace=True)
st.write(appSess_df)
#appSess_df2 = appSess_df.loc[appSess_df['USER_NAME'].str.contains('PUBLIC')]
appSess_df2 = appSess_df.loc[appSess_df['TOTAL_SESSIONS_sum'] > 1]



df = pd.DataFrame(appSess_df2)

# 2. Prepare data for Sankey diagram
# Create a list of unique labels from 'Source_Category' and 'Target_Category'
labels = list(pd.concat([df['USER_NAME'], df['APP'],df['QUERY_TYPE']]).unique())


# Map labels to indices, as Plotly's Sankey expects numerical indices for source and target
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create source, target, and value lists based on the DataFrame
sources = df['USER_NAME'].map(label_to_index).tolist()
sources = sources + df['APP'].map(label_to_index).tolist()
targets = df['APP'].map(label_to_index).tolist()
targets = targets + df['QUERY_TYPE'].map(label_to_index).tolist()
values = df['TOTAL_SESSIONS_sum'].tolist()
values = values + df['TOTAL_SESSIONS_sum'].tolist()
#st.write(sources)
#st.write(targets)
#st.write(values)

# 3. Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(     
    node=dict(
        pad=20,
        thickness=100,
        line=dict(color="black", width=0.5),
        label=labels,  # Use the unique labels for node labels
        
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        label=labels,
        
    )
)])

fig.update_layout(title_text="User Sessions vs. App , vs. Query Type",
                  title_font_color="Black",
                  font=dict(size = 15, color = "Black",family="Arial"),)
                 #plot_bgcolor='black',
                 #paper_bgcolor='black')
# fig.update_layout(
#     font=dict(
#         family="Arial, sans-serif",  # Specify a font family or preference list
#         size=14,  # Adjust font size
#         color="black"  # Set font color
#     )
# )
st.plotly_chart(fig)


In [ ]:
select
    -- query_history fields
    qh.query_id, qh.query_type, left(qh.query_text,200) query_text, qh.query_parameterized_hash,
    qh.warehouse_id, qh.warehouse_name, qh.warehouse_size, qh.warehouse_type, qh.cluster_number,
    qh.query_load_percent, qh.query_acceleration_upper_limit_scale_factor,
    qh.partitions_scanned, qh.query_acceleration_partitions_scanned, qh.bytes_scanned, qh.bytes_written,
    qh.bytes_spilled_to_local_storage, qh.bytes_spilled_to_remote_storage,
    qh.start_time, qh.end_time, qh.total_elapsed_time, qh.execution_time, qh.queued_overload_time,
    qh.error_code, qh.user_name,
    dateadd('millisecond',(
        queued_provisioning_time + queued_repair_time + queued_overload_time +
        transaction_blocked_time + compilation_time + list_external_files_time),
        qh.start_time) as execution_start_time,
    
    -- credit attribution
    qah.credits_attributed_compute, coalesce(qah.credits_used_query_acceleration,0) credits_used_query_acceleration,

    -- query sizing fields
    case when warehouse_type = 'STANDARD' THEN
      decode(warehouse_size, 'X-Small', 1, 'Small', 2, 'Medium', 4, 'Large', 8, 'X-Large', 16, '2X-Large', 32, '3X-Large', 64, '4X-Large', 128, '5X-Large', 64, '6X-Large', 128, null)
    else
      decode(warehouse_size, 'Medium', 1, 'Large', 2, 'X-Large', 4, '2X-Large', 8, '3X-Large', 16, '4X-Large', 32, '5X-Large', 16, '6X-Large', 32, null) -- are these spo node counts right?
    end as nodes_avail,
    partitions_scanned / ((nullif(query_load_percent,0)/100) * nodes_avail) as files_per_node,
    round((query_load_percent/100)*nodes_avail) as nodes_used,

    -- sizing rules
    -- case
    --     when files_per_vcpu < 8 then -2
    --     when files_per_vcpu < 64 then -1
    --     when files_per_vcpu > 2000 then +2
    --     when files_per_vcpu > 500 then +1
    --     else 0
    -- end as files_ind,
    case
        when files_per_node < 32 then -2
        when files_per_node < 256 then -1
        when files_per_node > 8000 then +2
        when files_per_node > 2000 then +1
        else 0
    end as files_ind,
    case
        when bytes_spilled_to_remote_storage > 0 then 2
        when bytes_spilled_to_local_storage > 0 then 1    -- for copy statements, bytes scanned might show up as spillage due to local caching
        else 0
    end as spill_ind,
    case when query_load_percent < 100 then -1 else 0 end as qlp_ind,
    -- case when query_acceleration_upper_limit_scale_factor > 0 then 1 else 0 end as qas_ind,
    case when exists (
        select query_id
        from pst.svcs.query_acceleration_eligible
        where query_id = qh.query_id
          and start_time >= $start_time
    ) then 1 else 0 end as qas_ind,
    files_ind+spill_ind+qlp_ind+qas_ind as size_indicator,
    case
        when warehouse_size = 'X-Small' then iff(size_indicator > 0,'OVERSIZED','WELL-SIZED')
        when nodes_used = 1 then 'SINGLE_NODE'
        when size_indicator < 0 then 'UNDERSIZED'
        when size_indicator = 0 then 'WELL-SIZED'
        when size_indicator > 0 then 'OVERSIZED'
    end as query_size,
    decode(query_size,'SINGLE_NODE',1,'UNDERSIZED',2,'WELL-SIZED',3,'OVERSIZED',4) as query_size_sort
    
from pst.svcs.query_history qh
left join pst.svcs.query_attribution_history qah on qh.query_id = qah.query_id and qah.start_time >= $start_time
where --qh.start_time >= $start_time
  --and qh.start_time < current_date
  qh.start_time > dateadd(day,-8,current_date)
  and qh.cluster_number is not null
  and qh.warehouse_size is not null
  and qh.execution_time > 0;

In [ ]:
select warehouse_size, query_type, query_size,
    round(sum(credits_attributed_compute+coalesce(credits_used_query_acceleration,0))) as credits,
    count(*) queries, count(distinct query_parameterized_hash) as query_hashes,
    round(avg(execution_time)/1000,1) avg_exec_sec,
    round(percentile_cont(.9) within group (order by execution_time)/1000,1) p90_exec_sec,
    round(avg(total_elapsed_time-execution_time)/1000,1) avg_other_sec,
    round(percentile_cont(.9) within group (order by total_elapsed_time-execution_time)/1000,1) p90_other_sec,
    round(avg(query_load_percent)) as avg_qlp,
    round(percentile_cont(.9) within group (order by query_load_percent)) p90_qlp,
    round(avg(files_per_node),1) avg_files_per_node,
    round(percentile_cont(.9) within group (order by files_per_node),1) p90_files_per_node,
    round(avg(bytes_spilled_to_local_storage + bytes_spilled_to_remote_storage)/1024/1024/1024,1) avg_spill_gb,
    round(percentile_cont(.9) within group (order by bytes_spilled_to_local_storage + bytes_spilled_to_remote_storage)/1024/1024/1024,1) p90_spill_gb,
    sum(qas_ind) as qas_queries
from {{cache_query_history}}
where warehouse_name = $warehouse_name
group by all
order by credits desc;

In [ ]:
select query_parameterized_hash, any_value(query_text) as sample_query_text,
    warehouse_size, query_type, query_size,
    round(sum(credits_attributed_compute+coalesce(credits_used_query_acceleration,0))) as credits,
    count(*) queries,
    round(avg(execution_time)/1000,1) avg_exec_sec,
    round(percentile_cont(.9) within group (order by execution_time)/1000,1) p90_exec_sec,
    round(avg(total_elapsed_time-execution_time)/1000,1) avg_other_sec,
    round(percentile_cont(.9) within group (order by total_elapsed_time-execution_time)/1000,1) p90_other_sec,
    round(avg(query_load_percent)) as avg_qlp,
    round(percentile_cont(.9) within group (order by query_load_percent)) p90_qlp,
    round(avg(files_per_node),1) avg_files_per_node,
    round(percentile_cont(.9) within group (order by files_per_node),1) p90_files_per_node,
    round(avg(bytes_spilled_to_local_storage + bytes_spilled_to_remote_storage)/1024/1024/1024,1) avg_spill_gb,
    round(percentile_cont(.9) within group (order by bytes_spilled_to_local_storage + bytes_spilled_to_remote_storage)/1024/1024/1024,1) p90_spill_gb,
    sum(qas_ind) as qas_queries,
    any_value(query_id) as sample_query_id
from {{cache_query_history}}
where warehouse_name = $warehouse_name
group by all
order by credits desc nulls last
limit 1000;

In [ ]:
select USER_NAME, ROLE_NAME, COUNT(DISTINCT SESSION_ID) AS TOTAL_SESSIONS
from
    pst.svcs.QUERY_HISTORY
where
    start_time >= dateadd('day', -30, current_timestamp())
    AND WAREHOUSE_NAME NOT ILIKE 'COMPUTE_%'
    Group by ALL;

In [ ]:
SessCount =  SessionsCounts.to_pandas()
st.bar_chart(SessCount, x="ROLE_NAME", y="TOTAL_SESSIONS", color="USER_NAME",horizontal = True)


# Create an Altair bar chart
chart = alt.Chart(SessCount).mark_bar(size=40).encode(
    x='ROLE_NAME',
    y='TOTAL_SESSIONS',
    color = 'USER_NAME'
)


# Display the chart in Streamlit
st.altair_chart(chart, use_container_width=True)